In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from pandas_datareader import data as pdr
yf.pdr_override() # <== that's all it takes :-)
import numpy as np
import statistics

sp500_history = pd.read_csv('./sp500_history.csv')
sp500_history['date'] = pd.to_datetime(sp500_history['date']) 
sp500_history = sp500_history[['date', 'value', 'variable']]

start_date = datetime.strptime('2016-1-1', '%Y-%m-%d')
end_date = datetime.strptime('2020-12-31', '%Y-%m-%d')

# greater than the start date and smaller than the end date
mask = (sp500_history['date'] > start_date) & (sp500_history['date'] <= end_date)
sp500_history = sp500_history.loc[mask]

# Sort
sp500_history.sort_values(['date', 'variable'], ascending=[True, True])

C:\Users\Yingchen\Anaconda3\envs\finance\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


,date,value,variable
261,2016-01-05,WLTW,added_ticker
260,2016-01-05,FOSL,removed_ticker
262,2016-01-19,EXR,added_ticker
263,2016-01-19,CB,removed_ticker
264,2016-02-01,FRT,added_ticker
...,...,...,...
453,2019-12-23,ZBRA,added_ticker
454,2019-12-23,STE,added_ticker
449,2019-12-23,AMG,removed_ticker
451,2019-12-23,MAC,removed_ticker


In [2]:
calendar = pd.read_excel('./Nasdaq_Trading_Calendar.xlsx', sheet_name=None)
rebal_dates = []
for year in calendar:
    sheet = calendar[year]
    mask = sheet['S&P Indexes Rebalance S&P 500, S&P 400, and S&P 600'] == 1
    year_rebal_dates = sheet.loc[mask]
    for date in year_rebal_dates['Date'].values:
        rebal_dates.append(date)


In [3]:
rebal_dates

[numpy.datetime64('2020-03-20T00:00:00.000000000'),
 numpy.datetime64('2020-06-19T00:00:00.000000000'),
 numpy.datetime64('2020-09-18T00:00:00.000000000'),
 numpy.datetime64('2020-12-18T00:00:00.000000000'),
 numpy.datetime64('2019-03-15T00:00:00.000000000'),
 numpy.datetime64('2019-06-21T00:00:00.000000000'),
 numpy.datetime64('2019-09-20T00:00:00.000000000'),
 numpy.datetime64('2019-12-20T00:00:00.000000000'),
 numpy.datetime64('2018-03-16T00:00:00.000000000'),
 numpy.datetime64('2018-06-15T00:00:00.000000000'),
 numpy.datetime64('2018-09-21T00:00:00.000000000'),
 numpy.datetime64('2018-12-21T00:00:00.000000000'),
 numpy.datetime64('2017-03-17T00:00:00.000000000'),
 numpy.datetime64('2017-06-16T00:00:00.000000000'),
 numpy.datetime64('2017-09-15T00:00:00.000000000'),
 numpy.datetime64('2017-12-15T00:00:00.000000000'),
 numpy.datetime64('2016-03-18T00:00:00.000000000'),
 numpy.datetime64('2016-06-17T00:00:00.000000000'),
 numpy.datetime64('2016-09-16T00:00:00.000000000'),
 numpy.datet

In [57]:
columns_to_create = ['original_date', 'total', 'up', 'count']

# output = pd.DataFrame(columns=columns_to_create)
# output.set_index(['date', 'add/delete'])

# output_add = pd.DataFrame(columns=columns_to_create)
# output_delete = pd.DataFrame(columns=columns_to_create)
regular_add = pd.DataFrame(columns=columns_to_create)
regular_delete = pd.DataFrame(columns=columns_to_create)
ad_hoc_add = pd.DataFrame(columns=columns_to_create)
ad_hoc_delete = pd.DataFrame(columns=columns_to_create)

# output_add['date'] = pd.to_datetime(output_add['date'])
# output_add.set_index('date')
# output_delete['date'] = pd.to_datetime(output_delete['date'])
# output_delete.set_index('date')

In [58]:
from math import isnan

MAX_DATE_EXTENSION = 10
days_bef_aft = 10 # the range of days before and after effective date to pull from Yahoo finance
last_date = sp500_history.iloc[0, sp500_history.columns.get_loc('date')]
last_type = sp500_history.iloc[0, sp500_history.columns.get_loc('variable')]
last_changes = []
is_begin = True
is_regular = False
last_is_regular = False

for idx, row in sp500_history.iterrows():
    
    # End date is exclusive, so need to increase by 1
    prices = pdr.get_data_yahoo(row['value'], start=row['date'] - timedelta(days=days_bef_aft), end=row['date'] + timedelta(days=days_bef_aft + 1))
    if len(prices) == 0:
        continue
        
    # Find the adjusted effective date 
    # which is one trading day before the effective (market opening) date
    adj_date = row['date'] - timedelta(days=1)
    date_extension = 0
    while not adj_date in prices.index and date_extension < MAX_DATE_EXTENSION:
        adj_date -= timedelta(days=1)
        date_extension += 1
    if date_extension >= MAX_DATE_EXTENSION:
        continue
    adj_date_close = prices.loc[adj_date]['Close']
    if isnan(adj_date_close):
        continue
    
    if adj_date in rebal_dates:
        is_regular = True
    else:
        is_regular = False
        
    # Find the entry date, which is one trading day before the adjusted effective date
    prev_date = adj_date - timedelta(days=1)
    date_extension = 0
    while not prev_date in prices.index and date_extension < MAX_DATE_EXTENSION:
        prev_date -= timedelta(days=1)
        date_extension += 1
    if date_extension >= MAX_DATE_EXTENSION:
        continue
    prev_close = prices.loc[prev_date]['Close']
    if isnan(prev_close):
        continue

    # Find the next opening price
#     next_date = row['date']
#     date_extension = 0
#     while not next_date in prices.index and date_extension < MAX_DATE_EXTENSION:
#         next_date += timedelta(days=1)
#         date_extension += 1
#     if date_extension >= MAX_DATE_EXTENSION:
#         continue    
#     next_open = prices.loc[next_date]['Open']

    price_change = adj_date_close - prev_close

    # Calculate mean and all
    if not is_begin and (last_date != adj_date or last_type != row['variable']):
#         print(last_changes)
        count = len(last_changes)
        up = len(list(filter(lambda x: (x >= 0), last_changes))) 
        
        data = [up, count]

        if last_type == 'added_ticker':
            if last_is_regular:
                regular_add.loc[last_date, 'up':'count'] = data
            else:
                ad_hoc_add.loc[last_date, 'up':'count'] = data

        else:
            if last_is_regular:
                regular_delete.loc[last_date, 'up':'count'] = data
            else:
                ad_hoc_delete.loc[last_date, 'up':'count'] = data

        total = 0
        last_changes = []
    
    # Still in the same date
    else:
        if is_begin:
            total = 0
            is_begin = False
        else:
            total = last_row['total']
    
    total += price_change
    
    data = {'total': total, 'original_date': row['date']}
            
    if row['variable'] == 'added_ticker':
        if is_regular:
            if adj_date not in regular_add.index:
                regular_add = regular_add.append(pd.DataFrame(data, index = [pd.to_datetime(adj_date)]), ignore_index=False)
            else:
                regular_add.loc[adj_date, 'total'] = total
            
        else:
            if adj_date not in ad_hoc_add.index:
                ad_hoc_add = ad_hoc_add.append(pd.DataFrame(data, index = [pd.to_datetime(adj_date)]), ignore_index=False)
            else:
                ad_hoc_add.loc[adj_date, 'total'] = total
                
    else:
        if is_regular:
            if adj_date not in regular_delete.index:
                regular_delete = regular_delete.append(pd.DataFrame(data, index = [pd.to_datetime(adj_date)]), ignore_index=False)
            else:
                regular_delete.loc[adj_date, 'total'] = total
            
        else:
            if adj_date not in ad_hoc_delete.index:
                ad_hoc_delete = ad_hoc_delete.append(pd.DataFrame(data, index = [pd.to_datetime(adj_date)]), ignore_index=False)
            else:
                ad_hoc_delete.loc[adj_date, 'total'] = total
                
    last_date = adj_date
    last_type = row['variable']
    last_changes.append(price_change)
    last_is_regular = is_regular
    
    if last_type == 'added_ticker':
        if is_regular:
            last_row = regular_add.loc[last_date, :]
        else:
            last_row = ad_hoc_add.loc[last_date, :]
    
    else:
        if is_regular:
            last_row = regular_delete.loc[last_date, :]
        else:
            last_row = ad_hoc_delete.loc[last_date, :]
        
regular_add.index.names = ['date']
ad_hoc_add.index.names = ['date']
regular_delete.index.names = ['date']
ad_hoc_delete.index.names = ['date']



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BRCM: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- PCP: Data doesn't exist for startDate = 1453392000, endDate = 1455206400
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 c

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- WFM: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- Q: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[***

In [50]:
row['date']

Timestamp('2017-03-16 00:00:00')

In [51]:
price_change

47.83984375

In [52]:
adj_date_close

3330.139892578125

In [53]:
adj_date

Timestamp('2017-03-15 00:00:00')

In [54]:
prices

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-03-06,3119.620117,3196.169922,3090.909912,3186.600098,3186.600098,647158
2017-03-07,3196.169922,3205.739990,3157.889893,3157.889893,3157.889893,502227
2017-03-08,3157.889893,3253.590088,3157.889893,3186.600098,3186.600098,1304619
2017-03-09,3186.600098,3272.729980,3186.600098,3244.020020,3244.020020,327346
2017-03-10,3244.020020,3253.590088,3205.739990,3205.739990,3205.739990,1065293
2017-03-13,3167.459961,3234.449951,3157.889893,3167.459961,3167.459961,974723
2017-03-14,3157.889893,3387.560059,3157.889893,3282.300049,3282.300049,2161990
2017-03-15,3320.570068,3387.560059,3291.870117,3330.139893,3330.139893,181129
2017-03-16,3368.419922,3397.129883,3311.000000,3311.000000,3311.000000,854851


In [56]:
is_regular

False

In [25]:
statistics.stdev(regular_add['total'])

nan

In [26]:
regular_add['total']

0          NaN
1     0.229992
2          NaN
3     4.969994
4          NaN
5     0.250000
6          NaN
7     3.409996
8          NaN
9    -1.169998
10   -1.820000
11         NaN
12   -0.740005
13    0.809998
14         NaN
Name: total, dtype: float64

In [59]:
'''
Calculate population statistics
for regular add
'''
regular_add_sum = regular_add.sum(axis=0)
regular_add_sum['mean'] = regular_add_sum['total'] / regular_add_sum['count']
if regular_add_sum['count'] > 1:
    regular_add_sum['std'] = statistics.stdev(regular_add['total'])
else:
    regular_add_sum['std'] = 0
regular_add_sum['max'] = max(regular_add['total'])
regular_add_sum['min'] = max(regular_add['total'])
if regular_add_sum['std'] == 0:
    regular_add_sum['sharpe'] = regular_add_sum['mean'] / 0.00001 * np.sqrt(252)
else:
    regular_add_sum['sharpe'] = regular_add_sum['mean'] / regular_add_sum['std'] * np.sqrt(252)
regular_add_sum['uppct'] = regular_add_sum['up'] / regular_add_sum['count']


In [60]:
'''
Writing to excel
'''
writer = pd.ExcelWriter('sp500_regular_add.xlsx', engine='xlsxwriter')
regular_add.to_excel(writer, sheet_name='Data')
regular_add_sum.to_excel(writer, sheet_name='Population Statistics')
writer.save()

ad_hoc_add.to_csv('sp500_ad_hoc_add.csv')
regular_delete.to_csv('sp500_regular_delete.csv')
ad_hoc_delete.to_csv('sp500_ad_hoc_delete.csv')